<a href="https://colab.research.google.com/github/fjadidi2001/Denoise/blob/main/Denoise_May18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Step 1: Set Up Environment
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Install dependencies
!pip install torchvision opencv-python h5py kagglehub

In [12]:
import kagglehub
import os
import numpy as np
import random
import h5py
import torch
import torch.nn as nn
import torch.utils.data as udata
import cv2
import glob
from torch.nn.modules.loss import _Loss
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import argparse
import time
import math
import string
from PIL import Image
import matplotlib.pyplot as plt

In [4]:



# Download Pascal VOC 2012 dataset
print("Downloading Pascal VOC 2012 dataset...")
path = kagglehub.dataset_download("gopalbhattrai/pascal-voc-2012-dataset")
print("Path to dataset files:", path)

# Create directories
base_dir = '/content/drive/MyDrive/SSNet'
os.makedirs(base_dir, exist_ok=True)
os.makedirs(f'{base_dir}/data', exist_ok=True)
os.makedirs(f'{base_dir}/data/Kodak24', exist_ok=True) # For testing
os.makedirs(f'{base_dir}/logos', exist_ok=True)
os.makedirs(f'{base_dir}/saved_models', exist_ok=True)
os.makedirs(f'{base_dir}/ckpts', exist_ok=True)
os.makedirs(f'{base_dir}/output', exist_ok=True)

# Step 2: Define Utility Functions (utils.py equivalent)
def seed_torch(seed=1029):
 random.seed(seed)
 os.environ['PYTHONHASHSEED'] = str(seed)
 np.random.seed(seed)
 torch.manual_seed(seed)
 torch.cuda.manual_seed(seed)
 torch.cuda.manual_seed_all(seed)
 torch.backends.cudnn.benchmark = False
 torch.backends.cudnn.deterministic = True

def weights_init_kaiming(m):
 classname = m.__class__.__name__
 if classname.find('Conv') != -1:
 nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
 elif classname.find('Linear') != -1:
 nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
 elif classname.find('BatchNorm') != -1:
 m.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).clamp_(-0.025, 0.025)
 nn.init.constant_(m.bias.data, 0.0)

def batch_PSNR(img, imclean, data_range):
 Img = img.data.cpu().numpy().astype(np.float32)
 Iclean = imclean.data.cpu().numpy().astype(np.float32)
 PSNR = 0
 for i in range(Img.shape[0]):
 PSNR += peak_signal_noise_ratio(Iclean[i, :, :, :], Img[i, :, :, :], data_range=data_range)
 return PSNR / Img.shape[0]

def data_augmentation(image, mode):
 out = np.transpose(image, (1, 2, 0))
 if mode == 0:
 out = out
 elif mode == 1:
 out = np.flipud(out)
 elif mode == 2:
 out = np.rot90(out)
 elif mode == 3:
 out = np.rot90(out)
 out = np.flipud(out)
 elif mode == 4:
 out = np.rot90(out, k=2)
 elif mode == 5:
 out = np.rot90(out, k=2)
 out = np.flipud(out)
 elif mode == 6:
 out = np.rot90(out, k=3)
 elif mode == 7:
 out = np.rot90(out, k=3)
 out = np.flipud(out)
 return np.transpose(out, (2, 0, 1))

def add_text_noise(noise, occupancy=50):
 font = cv2.FONT_HERSHEY_SIMPLEX
 h, w, _ = noise.shape
 img_for_cnt = np.zeros((h, w), np.uint8)
 occupancy = np.random.uniform(0, occupancy)
 while True:
 n = random.randint(5, 10)
 random_str = ''.join([random.choice(string.ascii_letters + string.digits) for i in range(n)])
 font_scale = np.random.uniform(0.5, 1)
 thickness = random.randint(1, 3)
 (fw, fh), baseline = cv2.getTextSize(random_str, font, font_scale, thickness)
 x = random.randint(0, max(0, w - 1 - fw))
 y = random.randint(fh, h - 1 - baseline)
 color = (random.random(), random.random(), random.random())
 cv2.putText(noise, random_str, (x, y), font, font_scale, color, thickness)
 cv2.putText(img_for_cnt, random_str, (x, y), font, font_scale, 255, thickness)
 if (img_for_cnt > 0).sum() > h * w * occupancy / 100:
 break
 return noise

def add_watermark_noise(img_train, scale_lists=None, idx_lists=None, is_test=False, threshold=50):
 watermarks = []
 logo_dir = f'{base_dir}/logos'
 for ii in range(12):
 logo_path = os.path.join(logo_dir, f'{ii+1:02d}.png')
 if os.path.exists(logo_path):
 watermarks.append(Image.open(logo_path))
 else:
 print(f"Warning: Logo {logo_path} not found")
 watermarks.append(Image.new('RGBA', (100, 100)))

 img_train = img_train.numpy()
 imgn_train = img_train.copy()
 _, _, img_h, img_w = img_train.shape
 img_train = np.transpose(img_train, (0, 2, 3, 1))
 imgn_train = np.transpose(imgn_train, (0, 2, 3, 1))
 ans_scale_lists = scale_lists if scale_lists else []
 ans_idx_lists = idx_lists if idx_lists else []

 for i in range(len(img_train)):
 tmp = Image.fromarray((img_train[i] * 255).astype(np.uint8))
 img_for_cnt = Image.fromarray(np.zeros((img_h, img_w, 3), np.uint8))

 if scale_lists is None:
 scale_list = []
 idx = random.randint(0, len(watermarks)-1)
 ans_idx_lists.append(idx)
 watermark = watermarks[idx]
 w, h = watermark.size
 mark_size = np.array(watermark).size
 occupancy = threshold if is_test else np.random.uniform(0, 10)
 cnt, ratio = 0, img_w * img_h * 3 * occupancy / 100
 finish = False
 while True:
 if (ratio - cnt) < mark_size * 0.3:
 img_train[i] = np.array(tmp).astype(np.float64) / 255.
 break
 elif (ratio - cnt) < mark_size:
 scale = (ratio - cnt) * 1.0 / mark_size
 finish = True
 else:
 scale = np.random.uniform(0.5, 1)
 scale_list.append(scale)

 water = watermark.resize((int(w * scale), int(h * scale)))
 x = random.randint(0, img_w - int(w * scale))
 y = random.randint(0, img_h - int(h * scale))
 tmp.paste(water, (x, y), water)
 img_for_cnt.paste(water, (x, y), water)
 img_cnt = np.array(img_for_cnt)
 cnt = (img_cnt > 0).sum()
 if finish:
 img_train[i] = np.array(tmp).astype(np.float64) / 255.
 break
 ans_scale_lists.append(scale_list)
 else:
 scale_list = scale_lists[i]
 idx = idx_lists[i]
 watermark = watermarks[idx]
 w, h = watermark.size
 for ii in range(len(scale_list)):
 scale = scale_list[ii]
 water = watermark.resize((int(w * scale), int(h * scale)))
 x = random.randint(0, img_w - int(w * scale))
 y = random.randint(0, img_h - int(h * scale))
 tmp.paste(water, (x, y), water)
 img_train[i] = np.array(tmp).astype(np.float64) / 255.

 img_train = np.transpose(img_train, (0, 3, 1, 2))
 imgn_train = np.transpose(imgn_train, (0, 3, 1, 2))
 return img_train, img_train - imgn_train, ans_scale_lists, ans_idx_lists

def add_watermark_noise_test(img, num_wm=1):
 watermarks = []
 logo_dir = f'{base_dir}/logos'
 for ii in range(12):
 logo_path = os.path.join(logo_dir, f'{ii+1:02d}.png')
 if os.path.exists(logo_path):
 watermarks.append(Image.open(logo_path))
 else:
 watermarks.append(Image.new('RGBA', (100, 100)))

 img = img.numpy()
 imgn = img.copy()
 _, _, img_h, img_w = img.shape
 img = np.transpose(img, (0, 2, 3, 1))
 imgn = np.transpose(imgn, (0, 2, 3, 1))

 for i in range(len(img)):
 tmp = Image.fromarray((img[i] * 255).astype(np.uint8))
 idx = random.randint(0, len(watermarks)-1)
 watermark = watermarks[idx]
 w, h = watermark.size
 for ii in range(num_wm):
 scale = np.random.uniform(0.5, 1)
 water = watermark.resize((int(w * scale), int(h * scale)))
 x = random.randint(0, img_w - int(w * scale))
 y = random.randint(0, img_h - int(h * scale))
 tmp.paste(water, (x, y), water)
 img[i] = np.array(tmp).astype(np.float64) / 255.

 img = np.transpose(img, (0, 3, 1, 2))
 imgn = np.transpose(imgn, (0, 3, 1, 2))
 return img, img - imgn

def findLastCheckpoint(save_path):
 files = glob.glob(os.path.join(save_path, '*.pth'))
 last_epoch = -1
 for fi in files:
 epoch = int(os.path.basename(fi)[-7:-4])
 if last_epoch < epoch:
 last_epoch = epoch
 return last_epoch

# Step 3: Define Data Preparation
def normalize(data):
 return data / 255.

def Im2Patch(img, win, stride=1):
 k = 0
 endc = img.shape[0]
 endw = img.shape[1]
 endh = img.shape[2]
 patch = img[:, 0:endw-win+0+1:stride, 0:endh-win+0+1:stride]
 TotalPatNum = patch.shape[1] * patch.shape[2]
 Y = np.zeros([endc, win*win, TotalPatNum], np.float32)
 for i in range(win):
 for j in range(win):
 patch = img[:, i:endw-win+i+1:stride, j:endh-win+j+1:stride]
 Y[:, k, :] = np.array(patch[:]).reshape(endc, TotalPatNum)
 k = k + 1
 return Y.reshape([endc, win, win, TotalPatNum])

def prepare_data(data_path, voc_path, patch_size, stride, aug_times=1, mode='color'):
 print('Process training data')
 scales = [1, 0.9, 0.8, 0.7]

 # Get all JPEG images from VOC2012/JPEGImages
 files = glob.glob(os.path.join(voc_path, 'VOC2012/JPEGImages', '*.jpg'))
 files.sort()

 # Split into train (80%) and validation (20%)
 random.seed(42)
 random.shuffle(files)
 split_idx = int(0.8 * len(files))
 train_files = files[:split_idx]
 val_files = files[split_idx:]

 # Process training data
 h5f = h5py.File(os.path.join(data_path, 'train_color_right.h5'), 'w')
 train_num = 0
 for i in range(len(train_files)):
 img = cv2.imread(train_files[i])
 h, w, c = img.shape
 for k in range(len(scales)):
 if int(h * scales[k]) < 256 or int(w * scales[k]) < 256:
 continue
 Img = cv2.resize(img, (int(h * scales[k]), int(w * scales[k])), interpolation=cv2.INTER_CUBIC)
 Img = np.transpose(Img, (2, 0, 1))
 Img = np.float32(normalize(Img))
 patches = Im2Patch(Img, win=patch_size, stride=stride)
 for n in range(patches.shape[3]):
 data = patches[:, :, :, n].copy()
 h5f.create_dataset(str(train_num), data=data)
 train_num += 1
 for m in range(aug_times - 1):
 data_aug = data_augmentation(data, np.random.randint(1, 8))
 h5f.create_dataset(str(train_num) + "_aug_%d" % (m + 1), data=data_aug)
 train_num += 1
 h5f.close()

 # Process validation data
 print('\nProcess validation data')
 h5f = h5py.File(os.path.join(data_path, 'val_color_right.h5'), 'w')
 val_num = 0
 for i in range(len(val_files)):
 img = cv2.imread(val_files[i])
 img = np.transpose(img, (2, 0, 1))
 img = np.float32(normalize(img))
 h5f.create_dataset(str(val_num), data=img)
 val_num += 1
 h5f.close()
 print(f'Training set, # samples {train_num}')
 print(f'Validation set, # samples {val_num}')

# Step 4: Define Dataset
class Dataset(udata.Dataset):
 def __init__(self, train=True):
 super(Dataset, self).__init__()
 self.train = train
 data_path = f'{base_dir}/data'
 h5f = h5py.File(os.path.join(data_path, 'train_color_right.h5'), 'r') if train else h5py.File(os.path.join(data_path, 'val_color_right.h5'), 'r')
 self.keys = list(h5f.keys())
 random.shuffle(self.keys)
 h5f.close()

 def __len__(self):
 return len(self.keys)

 def __getitem__(self, index):
 data_path = f'{base_dir}/data'
 h5f = h5py.File(os.path.join(data_path, 'train_color_right.h5'), 'r') if self.train else h5py.File(os.path.join(data_path, 'val_color_right.h5'), 'r')
 key = self.keys[index]
 data = np.array(h5f[key])
 h5f.close()
 return torch.Tensor(data)

# Step 5: Define Models
class sum_squared_error(_Loss):
 def __init__(self, size_average=None, reduce=None, reduction='sum'):
 super(sum_squared_error, self).__init__(size_average, reduce, reduction)

 def forward(self, input, target):
 return torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='sum').div_(2)

class UNet_Atten_3(nn.Module):
 def __init__(self, in_channels=3, out_channels=3):
 super(UNet_Atten_3, self).__init__()
 self._block1_dw = nn.Sequential(
 nn.Conv2d(in_channels, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(48, 48, 3, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block2_dw = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block3_dw = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(48, 48, 3, stride=2, padding=1, output_padding=1))
 self._block4_dw = nn.Sequential(
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block5_dw = nn.Sequential(
 nn.Conv2d(144, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block6_dw = nn.Sequential(
 nn.Conv2d(96 + in_channels, 64, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(64, 32, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(32, out_channels, 3, stride=1, padding=1),
 nn.LeakyReLU(0.1))
 self._block1_dw2 = nn.Sequential(
 nn.Conv2d(in_channels, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(48, 48, 3, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block2_dw2 = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block3_dw2 = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(48, 48, 3, stride=2, padding=1, output_padding=1))
 self._block4_dw2 = nn.Sequential(
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block5_dw2 = nn.Sequential(
 nn.Conv2d(144, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block6_dw2 = nn.Sequential(
 nn.Conv2d(96 + in_channels, 64, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(64, 32, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(32, out_channels, 3, stride=1, padding=1),
 nn.LeakyReLU(0.1))
 self._block1_dn = nn.Sequential(
 nn.Conv2d(in_channels, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(48, 48, 3, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block2_dn = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block3_dn = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(48, 48, 3, stride=2, padding=1, output_padding=1))
 self._block4_dn = nn.Sequential(
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block5_dn = nn.Sequential(
 nn.Conv2d(144, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block6_dn = nn.Sequential(
 nn.Conv2d(96 + in_channels, 64, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(64, 32, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(32, out_channels, 3, stride=1, padding=1),
 nn.LeakyReLU(0.1))
 self._block1_wm = nn.Sequential(
 nn.Conv2d(in_channels, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(48, 48, 3, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block2_wm = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(2))
 self._block3_wm = nn.Sequential(
 nn.Conv2d(48, 48, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(48, 48, 3, stride=2, padding=1, output_padding=1))
 self._block4_wm = nn.Sequential(
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block5_wm = nn.Sequential(
 nn.Conv2d(144, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(96, 96, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(96, 96, 3, stride=2, padding=1, output_padding=1))
 self._block6_wm = nn.Sequential(
 nn.Conv2d(96 + in_channels, 64, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(64, 32, 3, stride=1, padding=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(32, out_channels, 3, stride=1, padding=1),
 nn.LeakyReLU(0.1))

 self.avg_dn = nn.AdaptiveAvgPool2d((1, 1))
 self.attn_dn = nn.Sequential(
 nn.Conv2d(144, 6, kernel_size=1, stride=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(6, 144, kernel_size=1, stride=1),
 nn.Sigmoid())
 self.avg_wm = nn.AdaptiveAvgPool2d((1, 1))
 self.attn_wm = nn.Sequential(
 nn.Conv2d(144, 6, kernel_size=1, stride=1),
 nn.ReLU(inplace=True),
 nn.Conv2d(6, 144, kernel_size=1, stride=1),
 nn.Sigmoid())
 self.out = nn.Sequential(
 nn.Conv2d(out_channels * 2, out_channels, kernel_size=3, stride=1, padding=1),
 nn.LeakyReLU(0.1))

 self._init_weights()

 def _init_weights(self):
 for m in self.modules():
 if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
 nn.init.kaiming_normal_(m.weight.data)
 m.bias.data.zero_()

 def forward(self, x):
 pool1_dn = self._block1_dn(x)
 pool2_dn = self._block2_dn(pool1_dn)
 pool3_dn = self._block2_dn(pool2_dn)
 pool4_dn = self._block2_dn(pool3_dn)
 pool5_dn = self._block2_dn(pool4_dn)
 upsample5_dn = self._block3_dn(pool5_dn)
 concat5_dn = torch.cat((upsample5_dn, pool4_dn), dim=1)
 upsample4_dn = self._block4_dn(concat5_dn)
 concat4_dn = torch.cat((upsample4_dn, pool3_dn), dim=1)
 upsample3_dn = self._block5_dn(concat4_dn)
 concat3_dn = torch.cat((upsample3_dn, pool2_dn), dim=1)
 upsample2_dn = self._block5_dn(concat3_dn)
 concat2_dn = torch.cat((upsample2_dn, pool1_dn), dim=1)
 upsample1_dn = self._block5_dn(concat2_dn)
 concat1_dn = torch.cat((upsample1_dn, x), dim=1)
 out_denoise = self._block6_dn(concat1_dn)

 pool1_wm = self._block1_wm(out_denoise)
 pool2_wm = self._block2_wm(pool1_wm)
 pool3_wm = self._block2_wm(pool2_wm)
 pool4_wm = self._block2_wm(pool3_wm)
 pool5_wm = self._block2_wm(pool4_wm)
 upsample5_wm = self._block3_wm(pool5_wm)
 concat5_wm = torch.cat((upsample5_wm, pool4_wm), dim=1)
 upsample4_wm = self._block4_wm(concat5_wm)
 concat4_wm = torch.cat((upsample4_wm, pool3_wm), dim=1)
 upsample3_wm = self._block5_wm(concat4_wm)
 concat3_wm = torch.cat((upsample3_wm, pool2_wm), dim=1)
 upsample2_wm = self._block5_wm(concat3_wm)
 concat2_wm = torch.cat((upsample2_wm, pool1_wmphysics/chemistry practicals for class 11 & 12 VGS for CSE, NEET, JEE, CBSE, ICSE, Karnataka (PUC), National, International, IGCSE & IB Boards - Free Study Material, Solved Sample Question Papers & Practice Questions, Online Study Resources for Class 1 to 12 | VGS Vidyashram School - VGS Global School pool1_dw = self._block1_dw(x)
 pool2_dw = self._block2_dw(pool1_dw)
 pool3_dw = self._block2_dw(pool2_dw)
 pool4_dw = self._block2_dw(pool3_dw)
 pool5_dw = self._block2_dw(pool4_dw)
 upsample5_dw = self._block3_dw(pool5_dw)
 concat5_dw = torch.cat((upsample5_dw, pool4_dw), dim=1)
 upsample4_dw = self._block4_dw(concat5_dw)
 concat4_dw = torch.cat((upsample4_dw, pool3_dw), dim=1)
 upsample3_dw = self._block5_dw(concat4_dw)
 concat3_dw = torch.cat((upsample3_dw, pool2_dw), dim=1)
 upsample2_dw = self._block5_dw(concat3_dw)
 concat2_dw = torch.cat((upsample2_dw, pool1_dw), dim=1)
 mid_dn = self.avg_dn(concat2_dn)
 Scale_dn = self.attn_dn(mid_dn)
 concat2_dw = concat2_dw * Scale_dn
 upsample1_dw = self._block5_dw(concat2_dw)
 concat1_dw = torch.cat((upsample1_dw, x), dim=1)
 main_out_mid = self._block6_dw(concat1_dw)

 pool1_dw2 = self._block1_dw2(main_out_mid)
 pool2_dw2 = self._block2_dw2(pool1_dw2)
 pool3_dw2 = self._block2_dw2(pool2_dw2)
 pool4_dw2 = self._block2_dw(pool3_dw2)
 pool5_dw2 = self._block2_dw(pool4_dw2)
 upsample5_dw2 = self._block3_dw(pool5_dw2)
 concat5_dw2 = torch.cat((upsample5_dw2, pool4_dw2), dim=1)
 upsample4_dw2 = self._block4_dw(concat5_dw2)
 concat4_dw2 = torch.cat((upsample4_dw2, pool3_dw2), dim=1)
 upsample3_dw2 = self._block5_dw(concat4_dw2)
 concat3_dw2 = torch.cat((upsample3_dw2, pool2_dw2), dim=1)
 upsample2_dw2 = self._block5_dw(concat3_dw2)
 concat2_dw2 = torch.cat((ups  Free Study Material, Solved Sample Question Papers & Practice Questions for Class 1 to 12 | VGS Vidyashram School - VGS Global School | Online Study Resources | Free Study Material, Solved Sample Question Papers & Practice Questions for Class 1 to 12 | VGS Vidyashram School - VGS Global School | Online Study Resources | Free Study Material, Solved Sample Question Papers & Practice Questions for Class 1 to 12 | VGS Vidyashram School - VGS Global School | Online Study Resources | Free Study Material, Solved Sample Question Papers & Practice Questions for Class 1 to 12 | VGS Vidyashram School - VGS Global School | Online Study Resources | Free Study Material, Solved Sample Question Papers & Practice Questions for Class 1 to 12 | VGS Vidyashram School - VGS

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully un